In [1]:
!pip install paho-mqtt


     |████████████████████████████████| 102kB 10.5MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.0-cp36-none-any.whl size=61416 sha256=6857ca3238e8272ce4549213fd80e874355835052ad746e04d1b80bdaf139493
  Stored in directory: /root/.cache/pip/wheels/02/94/6c/8474137cb7a5a3e001d70a22c8ff919caee69435376bccce79
Successfully built paho-mqtt


In [0]:
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder,StandardScaler,OrdinalEncoder
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#import tensorflow as tf
from sklearn.model_selection import GridSearchCV , cross_val_score
from sklearn.metrics import  accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,roc_curve,classification_report
from termcolor import colored
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import SVC
#from catboost import Pool, CatBoostClassifier, cv
import paho.mqtt.client as mqtt
import json
import csv
import os
import ast

In [0]:
def read_dataset(path):
    return pd.read_csv(path)

class DataFrameSelector (BaseEstimator, TransformerMixin):
    def __init__ (self,attribute_names):
        self.attribute_names = attribute_names
    def fit(self,X,y = None):
        return self
    def transform(self,X,y = None):
        return X[self.attribute_names].values
    
class LabelBinarizerPipeline(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, X, y=None):
        self.encoder.fit(X)
        return self
    def transform(self, X, y=None):
        return self.encoder.transform(X)

In [0]:
sub_topic = "to_model"
pub_topic = "from_model"
pub_data = 'data'

data_point = [[]]
def on_connect(client, userdata, flags, rc):
    if (rc==0):
        print ("Connection Successfull!")
        client.subscribe(sub_topic)
        print ("Subscribe to '%s'"%sub_topic)
    else:
        print("Connected with result code "+str(rc))

        
def on_message(client, userdata, msg):
    global data_point
    data_point,dataset = get_data_point(msg.payload)
    print(data_point)
    res = int(rfc.predict(data_point)[0])
    print("Predicted Result: ",res)
    client.publish(pub_topic,str(res))
    client.publish(pub_data,dataset)

def get_data_point(data1):
    d_point = [[]]
    result = {'f0:ec:af:cf:6c:e1':-200, 'c2:b6:6e:70:fa:f7':-200,
       'd9:5f:f5:4f:10:89':-200, 'c4:52:32:5c:31:e7':-200, 'e9:3c:4a:34:13:fb':-200,
       'ed:61:e4:e8:22:30':-200, 'ea:01:26:75:a4:c3':-200, 'd0:4e:10:2e:cb:84':-200,
       'e4:e0:0a:ae:fd:e2':-200, 'fa:35:76:56:6f:e3':-200, 'd5:b7:dc:69:ca:ae':-200,
       'ca:81:7a:d7:55:49':-200, 'e7:2b:ea:2f:95:c5':-200, 'd4:32:fc:b5:f0:b5':-200}
    try:
        data = ast.literal_eval(data1.decode("utf-8"))
        #print(data)
        for key,value in data.items():
            if (value==-200):
                result[key] = "Not in Range"
            elif key in result and result[key] == -200:
                result[key] = float(value)
        #print(result)

        for key,value in result.items():
            if (value=="Not in Range"):
                d_point[0].append(-200)
            else:
                d_point[0].append(value)
        return d_point, json.dumps(result)
        
    except Exception as e:
        print(e)

In [0]:
df_train = read_dataset('drive/My Drive/IoT/ble_data.csv')

In [7]:
df_train.head()

,id,f0:ec:af:cf:6c:e1,c9:a6:4d:9b:c0:8c,c2:b6:6e:70:fa:f7,d9:5f:f5:4f:10:89,c4:52:32:5c:31:e7,e9:3c:4a:34:13:fb,ed:61:e4:e8:22:30,ea:01:26:75:a4:c3,d0:4e:10:2e:cb:84,e4:e0:0a:ae:fd:e2,fa:35:76:56:6f:e3,d5:b7:dc:69:ca:ae,ca:81:7a:d7:55:49,e7:2b:ea:2f:95:c5,d4:32:fc:b5:f0:b5
0,21.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
1,2.0,-200.0,-200.0,-200.0,-86.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
2,7.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
3,19.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-86.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-88.0
4,18.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-84.0,-74.0,-200.0


In [8]:
df_train.columns

Index(['id', 'f0:ec:af:cf:6c:e1', 'c9:a6:4d:9b:c0:8c', 'c2:b6:6e:70:fa:f7',
       'd9:5f:f5:4f:10:89', 'c4:52:32:5c:31:e7', 'e9:3c:4a:34:13:fb',
       'ed:61:e4:e8:22:30', 'ea:01:26:75:a4:c3', 'd0:4e:10:2e:cb:84',
       'e4:e0:0a:ae:fd:e2', 'fa:35:76:56:6f:e3', 'd5:b7:dc:69:ca:ae',
       'ca:81:7a:d7:55:49', 'e7:2b:ea:2f:95:c5', 'd4:32:fc:b5:f0:b5'],
      dtype='object')

In [0]:
df_train = df_train.drop(['c9:a6:4d:9b:c0:8c'],axis = 1)
X = df_train.iloc[:,1:].values
y = df_train.iloc[:,0].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state = 42)

In [10]:
rfc = RandomForestClassifier(max_depth=10,random_state=0)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
acc = accuracy_score(y_test,y_pred)
acc

0.8584229390681004

In [11]:
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("mqtt.eclipse.org", 1883, 60)
client.loop_forever()

Connection Successfull!
Subscribe to 'to_model'
[[-200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200]]
Predicted Result:  21
[[-200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200]]
Predicted Result:  21
Connection Successfull!
Subscribe to 'to_model'
[[-200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200]]
Predicted Result:  21
[[-200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200]]
Predicted Result:  21
[[-20.0, -90.0, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200, -200]]
Predicted Result:  7
[[-20.0, -90.0, -200, -200, -200, -200, -20.0, -20.0, -200, -200, -200, -200, -200, -200]]
Predicted Result:  9


KeyboardInterrupt: ignored